# Creating a Grid Integrated Plant Design Problem using IDAES Surrogates

## Import necessary packages
To begin, we need to import necessary packages. The primary packages required are:
* Pyomo : Modeling the optimization problem
* IDAES: Surrogate modeling capabilities
* Tensorflow: load trained Keras surrogates that represent market simulations

In [36]:
import json
import pandas as pd
import tensorflow

#import pyomo and its objects
from pyomo.environ import (
    ConcreteModel,
    Var,
    Constraint,
    Expression,
    NonNegativeReals,
    Block,
    value,
    Objective,
)
import pyomo.environ as pyo

# Import IDAES components
from idaes.core.util import to_json, from_json
from idaes.core.solvers import get_solver

#Import IDAES SurrogateBlock
from idaes.core.surrogate.keras_surrogate import KerasSurrogate
from idaes.core.surrogate.sampling.scaling import OffsetScaler
from idaes.core.surrogate.surrogate_block import SurrogateBlock

# Import simple rankine cycle flowsheet
import dispatches.case_studies.simple_rankine_cycle.simple_rankine_cycle as src

## Load Keras Surrogates
The first step towards assembling our optimization model is to load the pre-trained Keras neural networks (available in this DISPATCHES repository) as well as associated meta-data saved in corresponding .json files. This meta data contains input and output labels, scaling information, and the input bounds used during training.

The keras models themselves represent the results of market simulations using the Prescient Production Cost Modeling open-source package. We previously trained neural network models to predict the following outputs based on market inputs:
* Annual Revenue \[MM\$\]: (the total amount of revenue our generator earned per year)
* Annual Number of Generator Startups
* Annual Zone Output \[hours\]: (the amount of time our generator spent at different power output intervals). For this neural network, we trained 11 different zone outputs (off,0-10% of pmax,10-20% of pmax,...,90-100% of pmax)

For market inputs, our trianing data consisted of 8 primary attributes:
* p_max (the nameplate plant capacity)
* p_min_multipler (the minimum operating output as a fration of p_max)
* ramp_rate_multiplier
* minimum_up_time
* minimum_dn_multiplier
* marginal_cost
* fixed_run_cost
* startup_profile

In [37]:
# load training meta-data for each surrogate. this includes things like scaling 
# information, labels, and input bounds

with open("training_parameters_revenue.json", 'rb') as f:
    rev_data = json.load(f)

with open("training_parameters_zones.json", 'rb') as f:
    zone_data = json.load(f)

with open("training_parameters_nstartups.json", 'rb') as f:
    nstartups_data = json.load(f)

# unpack scaling information for revenue
xm_rev = rev_data['xm_inputs']
xstd_rev = rev_data['xstd_inputs']
zm_rev = rev_data['zm']
zstd_rev = rev_data['zstd']

# unpack scaling information for number of startups
xm_nstart = nstartups_data['xm_inputs']
xstd_nstart = nstartups_data['xstd_inputs']
zm_nstartups = nstartups_data['zm']
zstd_nstartups = nstartups_data['zstd']

# unpack scaling information for duration at different operating outputs
xm_zones = zone_data['xm_inputs']
xstd_zones = zone_data['xstd_inputs']
zm_zones = zone_data['zm']
zstd_zones = zone_data['zstd']

# load surrogates from keras directories
keras_revenue = tensorflow.keras.models.load_model('keras_models/revenue', compile=False)
keras_nstartups = tensorflow.keras.models.load_model('keras_models/nstartups', compile=False)
keras_zones = tensorflow.keras.models.load_model('keras_models/zones', compile=False)

## Create IDAES Keras Surrogate Objects

Now that we have our keras models loaded we can create `KerasSurrogate` objects using IDAES. These objects wrap our trained Keras Sequential models and store the additional training information we loaded such as scaling. The `KerasSurrogate` object provides a consistent interface for handling surrogate models in IDAES.

Once we have a `KerasSurrogate`, we can use it to build a mathematical model on a `SurrogateBlock`. This is shown later in this notebook.

In [38]:
# revenue Keras surrogate
# load extra meta-data for the revenue surrogate
input_labels = list('x'+str(i) for i in range(len(xm_rev)))
output_labels = ["z_rev"]
xmin = rev_data['xmin']
xmax = rev_data['xmax']

# create scaling objects for the neural network input and outputs
inputs_scaler = OffsetScaler(
    expected_columns=input_labels,
    offset_series = pd.Series(dict(zip(input_labels, xm_rev))),
    factor_series = pd.Series(dict(zip(input_labels, xstd_rev))),
)

outputs_scaler = OffsetScaler(
    expected_columns=output_labels,
    offset_series = pd.Series(dict(zip(output_labels, [zm_rev]))),
    factor_series = pd.Series(dict(zip(output_labels, [zstd_rev]))),
)

# store input bounds in a dictionary where keys correspond to input labels and values are 2-length tuples
# of the lower and upper bound respectively
input_bounds={input_labels[i]: (xmin[i], xmax[i]) for i in range(len(input_labels))}

# create the KerasSurrogate object using the keras neural network and associated training data
keras_revenue_surrogate = KerasSurrogate(
    keras_model=keras_revenue,
    input_labels=input_labels,
    output_labels=output_labels,
    input_bounds=input_bounds,
    input_scaler=inputs_scaler,
    output_scaler=outputs_scaler,
)

# nstartups Keras surrogate
input_labels = list('x'+str(i) for i in range(len(xm_nstart)))
output_labels = ['z_nstartup']
xmin = nstartups_data['xmin']
xmax = nstartups_data['xmax']

inputs_scaler = OffsetScaler(
    expected_columns=input_labels,
    offset_series = pd.Series(dict(zip(input_labels, xm_nstart))),
    factor_series = pd.Series(dict(zip(input_labels, xstd_nstart))),
)

outputs_scaler = OffsetScaler(
    expected_columns=output_labels,
    offset_series = pd.Series(dict(zip(output_labels,[zm_nstartups]))),
    factor_series = pd.Series(dict(zip(output_labels,[zstd_nstartups]))),
)

input_bounds={input_labels[i]: (xmin[i], xmax[i]) for i in range(len(input_labels))}

keras_nstartups_surrogate = KerasSurrogate(
    keras_model=keras_nstartups,
    input_labels=input_labels,
    output_labels=output_labels,
    input_bounds=input_bounds,
    input_scaler=inputs_scaler,
    output_scaler=outputs_scaler,
)

# zone hour power output Keras surrogate
input_labels = list('x'+str(i) for i in range(len(xm_zones)))
output_labels = list('z_zone'+str(i) for i in range(len(zm_zones)) )
xmin = zone_data['xmin']
xmax = zone_data['xmax']

inputs_scaler = OffsetScaler(
    expected_columns=input_labels,
    offset_series = pd.Series(dict(zip(input_labels, xm_zones))),
    factor_series = pd.Series(dict(zip(input_labels, xstd_zones))),
 )

outputs_scaler = OffsetScaler(
    expected_columns=output_labels,
    offset_series = pd.Series(dict(zip(output_labels, zm_zones))),
    factor_series = pd.Series(dict(zip(output_labels, zstd_zones))),
)

input_bounds={input_labels[i]: (xmin[i], xmax[i]) for i in range(len(input_labels))}

keras_zones_surrogate = KerasSurrogate(
    keras_model=keras_zones,
    input_labels=input_labels,
    output_labels=output_labels,
    input_bounds=input_bounds,
    input_scaler=inputs_scaler,
    output_scaler=outputs_scaler,
)

## Building the Rankine Cycle Model

We now build the rankine cycle model available from DISPATCHES. This cell creates a pyomo model and uses rankine cycle functions to construct a design flowsheet. More specifically, we construct a high level flowsheet to represent the plant design. We will later construct additional flowsheets that represent plant operation for different power outputs.

In [39]:
# rankine cycle parameters
heat_recovery = True
calc_boiler_eff = True
capital_payment_years = 5
plant_lifetime = 20
coal_price = 30 #$/ton

m = ConcreteModel()

# Create capex plant
m.cap_fs = src.create_model(
    heat_recovery=heat_recovery,
    capital_fs=True, 
    calc_boiler_eff=False,
)
src.set_inputs(m.cap_fs)
src.initialize_model(m.cap_fs)
src.close_flowsheet_loop(m.cap_fs)
src.add_capital_cost(m.cap_fs)

# capital cost (M$/yr)
cap_expr = m.cap_fs.fs.capital_cost / capital_payment_years

2023-05-19 15:05:24 [INFO] idaes.init.cap_fs.fs.boiler.control_volume: Initialization Complete
2023-05-19 15:05:24 [INFO] idaes.init.cap_fs.fs.boiler: Initialization Complete: optimal - Optimal Solution Found
2023-05-19 15:05:25 [INFO] idaes.init.cap_fs.fs.turbine: Initialization Complete: optimal - Optimal Solution Found
2023-05-19 15:05:25 [INFO] idaes.init.cap_fs.fs.pre_condenser.control_volume: Initialization Complete
2023-05-19 15:05:25 [INFO] idaes.init.cap_fs.fs.pre_condenser: Initialization Complete: optimal - Optimal Solution Found
2023-05-19 15:05:25 [INFO] idaes.init.cap_fs.fs.condenser.control_volume: Initialization Complete
2023-05-19 15:05:25 [INFO] idaes.init.cap_fs.fs.condenser: Initialization Complete: optimal - Optimal Solution Found
2023-05-19 15:05:25 [INFO] idaes.init.cap_fs.fs.bfw_pump.control_volume: Initialization Complete
2023-05-19 15:05:25 [INFO] idaes.init.cap_fs.fs.bfw_pump: Initialization Complete: optimal - Optimal Solution Found
2023-05-19 15:05:25 [INFO

## Build the grid market surrogates

We now create variables to represent the previously defined market inputs such as name-plate capacity, ramp rate, marginal cost, etc...

We use the IDAES `SurrogateBlock` to build the neural network constraints by using the previously creates `KerasSurrogate` objects. Notably, the build function automatically performs the input and output scaling because we provided the scaling information to each `KerasSurrogate`.

In [40]:
########################################
# Surrogate market inputs
########################################
m.pmax = Var(within=NonNegativeReals, bounds=(175, 450), initialize=400) #MW
m.pmin_multi = Var(within=NonNegativeReals, bounds=(0.15, 0.45), initialize=0.3)
m.ramp_multi = Var(within=NonNegativeReals, bounds=(0.5, 1.0), initialize=0.75)
m.min_up_time = Var(within=NonNegativeReals, bounds=(1.0, 16.0), initialize=4.0)
m.min_dn_multi = Var(within=NonNegativeReals, bounds=(0.5, 2.0), initialize=1.0)
m.marg_cst =  Var(within=NonNegativeReals, bounds=(5, 30), initialize=15)
m.no_load_cst =  Var(within=NonNegativeReals, bounds=(0, 2.5), initialize=1)
m.startup_cst = Var(within=NonNegativeReals, bounds=(0, 136), initialize=75)

m.pmax_con = Constraint(expr=m.pmax == m.cap_fs.fs.net_cycle_power_output * 1e-6)

# Actual generator values for minimum operating output, minimum down time, and true ramp rate
m.pmin = Expression(expr=m.pmin_multi * m.pmax) # MW
m.min_dn_time = Expression(expr=m.min_dn_multi * m.min_up_time) # hours
m.ramp_rate = Expression(expr=m.ramp_multi * (m.pmax - m.pmin)) # MW/hr

In [41]:
######################################
# Revenue surrogate
######################################
m.rev_surrogate = Var()
m.keras_revenue_surrogate = SurrogateBlock()
m.keras_revenue_surrogate.build_model(
    keras_revenue_surrogate,
    input_vars=[
        m.pmax, m.pmin_multi, m.ramp_multi, m.min_up_time,
        m.min_dn_multi, m.marg_cst, m.no_load_cst, m.startup_cst,
    ],
    output_vars = [m.rev_surrogate],
    formulation=KerasSurrogate.Formulation.REDUCED_SPACE,
)

# this is a smooth-max; it sets negative revenue to zero
m.revenue = Expression(expr=0.5*pyo.sqrt(m.rev_surrogate**2 + 0.001**2) + 0.5*m.rev_surrogate)

In [42]:
#######################################
#nstartups surrogate
#######################################
m.nstartups_surrogate = Var()
m.keras_nstartups_surrogate = SurrogateBlock()
m.keras_nstartups_surrogate.build_model(
    keras_nstartups_surrogate,
    input_vars=[
        m.pmax, m.pmin_multi, m.ramp_multi, m.min_up_time,
        m.min_dn_multi, m.marg_cst, m.no_load_cst, m.startup_cst,
    ],
    output_vars = [m.rev_surrogate],
    formulation=KerasSurrogate.Formulation.REDUCED_SPACE,
)

m.nstartups = Expression(
    expr=0.5*pyo.sqrt(m.nstartups_surrogate**2 + 0.001**2) + 0.5*m.nstartups_surrogate
)

In [43]:
############################################
# zone surrogates
############################################
m.zone_hours_surrogate = Var(range(11))
m.keras_zones_surrogate = SurrogateBlock()
m.keras_zones_surrogate.build_model(
    keras_zones_surrogate,
    input_vars=[
        m.pmax,m.pmin_multi,m.ramp_multi,m.min_up_time,
        m.min_dn_multi,m.marg_cst,m.no_load_cst,m.startup_cst
    ],
    output_vars = m.zone_hours_surrogate,
    formulation=KerasSurrogate.Formulation.REDUCED_SPACE,
)

## Build the Operation Flowsheets

We now create a flowsheet for each possible operating zone. Each flowsheet uses the market decisions defined above and we use the zone surrogates to predict the number of hours spent at each flowsheet's powr output.

In [44]:
off_fs = Block()
off_fs.fs = Block()
off_fs.fs.operating_cost = m.no_load_cst * m.pmax
off_fs.zone_hours = Expression(
    expr=0.5*pyo.sqrt(m.zone_hours_surrogate[0]**2 + 0.001**2) + 0.5*m.zone_hours_surrogate[0]
)
setattr(m, 'zone_{}'.format('off'), off_fs)

# Denote the scaled power output for each of the 10 zones (0 corresponds to pmin, 1.0 corresponds to pmax)
zone_outputs = [0.0, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 1.0]

# Create a surrogate flowsheet for each operating zone
op_zones = []
init_flag = 0
for (i, zone_output) in enumerate(zone_outputs):
    print("Creating instance ", i)
    op_fs = src.create_model(
        heat_recovery=heat_recovery,
        capital_fs=False,
        calc_boiler_eff=calc_boiler_eff,
    )
    # Set model inputs for the capex and opex plant
    src.set_inputs(op_fs)

    # Fix the p_max of op_fs to p of cap_fs for initialization
    op_fs.fs.net_power_max.fix(value(m.cap_fs.fs.net_cycle_power_output))

    #initialize with json. this speeds up model instantiation. it writes a json file \
    #for the first flowsheet which is used to initialize the next flowsheets
    if init_flag == 0:
        # Initialize the opex plant
        src.initialize_model(op_fs)

        # save model state after initializing the first instance
        init_model = to_json(op_fs.fs, return_dict=True)
        init_flag = 1
    else:
        # Initialize the capex and opex plant
        from_json(op_fs.fs, sd=init_model)

    # Closing the loop in the flowsheet
    src.close_flowsheet_loop(op_fs)
    src.add_operating_cost(op_fs, coal_price=coal_price)

    # Unfix op_fs p_max and set constraint linking that to cap_fs p_max
    op_fs.fs.net_power_max.unfix()
    op_fs.fs.eq_p_max = Constraint(
        expr=op_fs.fs.net_power_max == m.cap_fs.fs.net_cycle_power_output * 1e-6
    )

    # Fix zone power output
    op_fs.fs.eq_fix_power = Constraint(
        expr=op_fs.fs.net_cycle_power_output * 1e-6 == zone_output * (m.pmax-m.pmin) + m.pmin
    )

    # smooth max on zone hours (avoids negative hours)
    op_fs.zone_hours = Expression(
        expr=0.5*pyo.sqrt(m.zone_hours_surrogate[i+1]**2 + 0.001**2) + 0.5*m.zone_hours_surrogate[i+1]
    )

    # unfix the boiler flow rate
    op_fs.fs.boiler.inlet.flow_mol[0].setlb(0.01)
    op_fs.fs.boiler.inlet.flow_mol[0].unfix()
    setattr(m, 'zone_{}'.format(i), op_fs)
    op_zones.append(op_fs)

# scale zone hours such that they add up to 8736 (if the surrogate is good, the unscaled will be pretty close to this)
m.zone_total_hours = sum(op_zones[i].zone_hours for i in range(len(op_zones))) + off_fs.zone_hours

for op_fs in op_zones:
    op_fs.scaled_zone_hours = Var(within=NonNegativeReals, bounds=(0, 8736), initialize=100)
    # NOTE: scaled_hours_i = surrogate_i * 8736 / surrogate_total
    op_fs.con_scale_zone_hours = Constraint(
        expr=op_fs.scaled_zone_hours * m.zone_total_hours == op_fs.zone_hours * 8736
    )

off_fs.scaled_zone_hours = Var(within=NonNegativeReals, bounds=(0, 8736), initialize=100)
off_fs.con_scale_zone_hours = Constraint(
    expr=off_fs.scaled_zone_hours * m.zone_total_hours == off_fs.zone_hours * 8736
)

#operating cost in $MM (million dollars)
m.op_expr = sum(
    op_zones[i].scaled_zone_hours * op_zones[i].fs.operating_cost 
    for i in range(len(op_zones))
) * 1e-6 + off_fs.scaled_zone_hours * off_fs.fs.operating_cost * 1e-6

#startup cost in MM$
m.startup_expr = m.startup_cst * m.nstartups * m.pmax * 1e-6 #MM$

# set zone flowsheets to pyomo model
m.op_zones = op_zones

# Piecewise cost limits, connect marginal cost to operating cost. We say marginal cost is the average operating cost
m.connect_mrg_cost = Constraint(
    expr=m.marg_cst == 0.5*(op_zones[0].fs.operating_cost/m.pmin + op_zones[-1].fs.operating_cost/m.pmax)
)

# Expression for total cap and op cost - $
m.total_cost = Expression(
    expr=plant_lifetime * (m.op_expr + m.startup_expr) + capital_payment_years*cap_expr
)

# Expression for total revenue
m.total_revenue = Expression(expr=plant_lifetime*m.revenue)

# Objective $
m.obj = Objective(expr=-(m.total_revenue - m.total_cost))

# Unfixing the boiler inlet flowrate for capex plant
m.cap_fs.fs.boiler.inlet.flow_mol[0].unfix()

# Setting bounds for the capex plant flowrate
m.cap_fs.fs.boiler.inlet.flow_mol[0].setlb(0.01)

# Setting bounds for net cycle power output for the capex plant
p_lower_bound=10
p_upper_bound=500
m.cap_fs.fs.eq_min_power = Constraint(
    expr=m.cap_fs.fs.net_cycle_power_output >= p_lower_bound*1e6)

m.cap_fs.fs.eq_max_power = Constraint(
    expr=m.cap_fs.fs.net_cycle_power_output <= p_upper_bound*1e6)

Creating instance  0
2023-05-19 15:05:39 [INFO] idaes.init.fs.boiler.control_volume: Initialization Complete
2023-05-19 15:05:39 [INFO] idaes.init.fs.boiler: Initialization Complete: optimal - Optimal Solution Found
2023-05-19 15:05:39 [INFO] idaes.init.fs.turbine: Initialization Complete: optimal - Optimal Solution Found
2023-05-19 15:05:39 [INFO] idaes.init.fs.pre_condenser.control_volume: Initialization Complete
2023-05-19 15:05:39 [INFO] idaes.init.fs.pre_condenser: Initialization Complete: optimal - Optimal Solution Found
2023-05-19 15:05:39 [INFO] idaes.init.fs.condenser.control_volume: Initialization Complete
2023-05-19 15:05:39 [INFO] idaes.init.fs.condenser: Initialization Complete: optimal - Optimal Solution Found
2023-05-19 15:05:39 [INFO] idaes.init.fs.bfw_pump.control_volume: Initialization Complete
2023-05-19 15:05:39 [INFO] idaes.init.fs.bfw_pump: Initialization Complete: optimal - Optimal Solution Found
2023-05-19 15:05:39 [INFO] idaes.init.fs.feed_water_heater.control_

## Setup Final Surrogate Inputs and Solve

We lastly fix a few surrogate inputs and solve the design problem. The solution of the design problem will determine market inputs that maximize the net plant revenue over 20 years.

In [45]:
# these are representative startup costs based on startup profiles we trained on.
# it is useful to fix the startup cost to one of these values since they are technically categorical variables
startup_csts = [0., 49.66991167, 61.09068702, 101.4374234,  135.2230393]

#fix some surrogate inputs
start_cst_index=2
m.startup_cst.fix(startup_csts[start_cst_index])
m.no_load_cst.fix(1.0)
m.min_up_time.fix(4.0)
m.min_dn_multi.fix(1.0)

solver = get_solver()
solver.options = {
    "tol": 1e-6
    #"mu_strategy": "adaptive"
}
status = solver.solve(m, tee=True)
sol_time = status['Solver'][0]['Time']

Ipopt 3.13.2: tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. S

 116r 5.4208187e+02 4.16e+03 2.32e+01  -0.8 3.07e+03    -  1.00e+00 9.02e-01f  1
 117r 4.4786163e+02 4.08e+03 3.27e+01  -0.8 2.05e+07    -  2.00e-01 1.38e-01f  1
 118r 4.4687357e+02 4.08e+03 1.25e+03  -0.8 1.76e+07    -  5.13e-01 1.68e-03f  1
 119r 4.2823182e+02 4.07e+03 2.53e+02  -0.8 1.76e+07    -  2.41e-03 3.20e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 120r 1.9208782e+02 1.15e+04 1.22e+02  -0.8 1.71e+07    -  4.66e-03 4.19e-01f  1
 121r 1.6262059e+02 1.06e+04 9.88e+01  -0.8 9.91e+06    -  3.25e-01 9.83e-02f  1
 122r 1.3902799e+02 8.87e+03 8.46e+01  -0.8 8.94e+06    -  6.19e-02 3.07e-01f  1
 123r 1.3052660e+02 7.74e+03 7.65e+01  -0.8 6.19e+06    -  2.97e-01 1.46e-01f  1
 124r 9.8669075e+01 5.41e+03 2.20e+02  -0.8 5.29e+06    -  1.00e+00 9.17e-01f  1
 125r 9.6418985e+01 3.59e+03 1.65e+01  -0.8 4.41e+05    -  1.00e+00 1.00e+00f  1
 126r 9.6202722e+01 3.59e+03 2.04e+00  -0.8 4.10e+02    -  1.00e+00 1.00e+00f  1
 127r 9.2251843e+01 3.59e+03

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 230r-1.5011774e+03 1.87e+05 9.99e+02   3.1 0.00e+00    -  0.00e+00 3.83e-07R  5
 231r-1.5003530e+03 1.09e+05 5.65e+03   3.1 2.36e+05    -  5.07e-05 1.56e-03f  1
 232r-1.5003514e+03 1.09e+05 9.36e+03   1.7 4.02e-01   4.0 8.01e-01 2.85e-01f  1
 233r-1.5003403e+03 1.09e+05 2.15e+03   1.7 5.35e-01   3.5 8.51e-01 6.23e-01f  1
 234r-1.5003040e+03 1.08e+05 1.20e+03   1.7 9.64e-01   3.0 1.00e+00 6.81e-01f  1
 235r-1.5002778e+03 1.08e+05 1.07e+03   1.7 2.73e+00   2.6 4.23e-01 1.52e-01f  1
 236r-1.5002500e+03 1.08e+05 3.69e+03   1.7 1.07e+00   3.0 1.00e+00 3.82e-01f  1
 237r-1.5002184e+03 1.08e+05 1.11e+03   1.7 4.46e-01   3.4 1.00e+00 1.00e+00f  1
 238r-1.5002145e+03 1.08e+05 2.95e+03   1.0 1.58e-01   3.8 9.35e-01 5.38e-01f  1
 239r-1.5002077e+03 1.08e+05 1.48e+03   1.0 4.87e-01   3.4 1.00e+00 2.77e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 240r-1.5001877e+03 1.07e+05

 326r-9.5482763e+02 1.52e+04 9.99e+02   2.8 0.00e+00    -  0.00e+00 3.89e-07R  3
 327r-9.4952721e+02 2.44e+03 3.00e+03   2.8 3.23e+05    -  7.70e-04 2.33e-02f  1
 328r-9.4952698e+02 2.43e+03 1.72e+04   1.4 3.15e-01   4.0 8.09e-01 1.00e+00f  1
 329r-9.4952541e+02 2.38e+03 7.63e+03   1.4 3.90e-01   3.5 1.00e+00 6.97e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 330r-9.4952312e+02 2.29e+03 5.22e+03   1.4 8.95e-01   3.0 1.00e+00 3.12e-01f  1
 331r-9.4951657e+02 2.06e+03 4.42e+03   1.4 2.67e+00   2.6 1.00e+00 2.19e-01f  1
 332r-9.4919525e+02 5.71e+02 4.39e+03   1.4 1.61e+06    -  9.39e-01 6.10e-03f  1
 333r-8.9466852e+02 1.24e+04 2.67e+03   1.4 1.60e+06    -  1.00e+00 1.00e+00f  1
 334r-8.9466852e+02 1.24e+04 9.99e+02   2.7 0.00e+00    -  0.00e+00 2.70e-07R  4
 335r-8.8965642e+02 1.98e+03 3.16e+03   2.7 3.50e+05    -  8.43e-04 2.41e-02f  1
 336r-8.8965637e+02 1.96e+03 1.18e+04   1.3 2.44e-01   4.0 9.12e-01 1.00e+00f  1
 337r-8.8965567e+02 1.93e+03

 426r 7.1608252e+01 4.29e+01 9.99e+02   1.6 0.00e+00    -  0.00e+00 6.22e-10R  2
 427r 7.2386798e+01 4.29e+01 5.80e+03   1.6 8.08e+03    -  1.68e-04 1.15e-02f  1
 428r 7.6156862e+01 4.29e+01 5.31e+03   1.6 8.00e+03    -  1.16e-01 6.72e-02f  1
 429r 7.6178957e+01 4.29e+01 4.70e+03   1.6 3.09e+00   2.0 1.39e-01 3.87e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 430r 7.6186595e+01 4.51e+01 3.54e+03   1.6 3.84e-01   3.3 5.93e-01 2.98e-01h  1
 431r 7.6248685e+01 9.47e+01 8.41e+03   1.6 4.19e-01   2.9 2.03e-01 7.78e-01h  1
 432r 9.2885485e+01 1.93e+03 7.88e+03   1.6 7.46e+03    -  8.98e-01 4.72e-01f  1
 433r 9.2901605e+01 1.06e+02 1.78e+03   1.6 2.45e-01   3.3 1.00e+00 1.00e+00f  1
 434r 9.2900575e+01 6.38e+01 1.03e+03   0.9 1.57e-01   3.7 4.99e-01 1.00e+00f  1
 435r 9.2901225e+01 4.29e+01 7.52e+02   0.9 1.16e-01   4.1 7.83e-01 1.00e+00f  1
 436r 9.0758846e+01 4.29e+01 6.17e+02   0.9 1.43e+04    -  2.17e-01 2.20e-01f  1
 437r 9.0098275e+01 4.29e+01

...

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1000  2.5444477e+01 1.94e+02 5.87e+09  -1.0 6.34e+08    -  1.00e+00 1.20e-04h 12
1001  2.5551351e+01 1.94e+02 5.87e+09  -1.0 6.34e+08    -  3.42e-02 1.20e-04h 12
1002  2.5657328e+01 1.94e+02 5.87e+09  -1.0 6.34e+08    -  9.94e-02 1.20e-04h 12
1003  2.4120885e+02 1.52e+06 4.40e+10  -1.0 6.34e+08    -  5.58e-02 2.45e-01w  1
1004  2.9534046e+02 1.52e+06 4.41e+10  -1.0 1.96e+08    -  7.07e-03 6.43e-03w  1
1005  3.2749732e+02 1.51e+06 4.62e+10  -1.0 1.95e+08    -  1.18e-02 3.76e-03w  1
1006  2.5762426e+01 1.94e+02 5.87e+09  -1.0 4.39e+08    -  5.58e-02 1.19e-04h 11
1007  2.5866638e+01 1.94e+02 5.87e+09  -1.0 6.34e+08    -  1.00e+00 1.19e-04h 12
1008  2.5969943e+01 1.95e+02 5.87e+09  -1.0 6.34e+08    -  3.41e-02 1.19e-04h 12
1009  2.6072353e+01 1.95e+02 5.87e+09  -1.0 6.34e+08    -  1.50e-01 1.19e-04h 12
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1010  2.6173857e+01 1.95e+02

1092  3.0906823e+01 2.04e+02 8.70e+09  -1.0 6.30e+08    -  5.79e-02 1.16e-04h 12
1093  3.0955027e+01 2.04e+02 8.70e+09  -1.0 6.29e+08    -  1.00e+00 1.16e-04h 12
1094  1.2933323e+02 1.43e+06 8.51e+10  -1.0 6.29e+08    -  4.77e-02 2.38e-01w  1
1095  1.4258458e+02 1.41e+06 8.40e+10  -1.0 2.04e+08    -  1.28e-02 1.28e-02w  1
1096  1.8268323e+02 1.40e+06 5.81e+10  -1.0 1.94e+08    -  3.09e-02 1.10e-02w  1
1097  3.1002494e+01 2.04e+02 8.70e+09  -1.0 1.93e+08    -  4.77e-02 1.16e-04h 11
1098  3.1049224e+01 2.04e+02 8.70e+09  -1.0 6.29e+08    -  1.57e-01 1.16e-04h 12
1099  3.1095223e+01 2.04e+02 8.70e+09  -1.0 6.29e+08    -  6.93e-02 1.16e-04h 12
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1100  3.1140489e+01 2.05e+02 8.70e+09  -1.0 6.29e+08    -  1.00e+00 1.16e-04h 12
1101  3.1184972e+01 2.05e+02 8.70e+09  -1.0 6.29e+08    -  4.91e-02 1.16e-04h 12
1102  3.1228725e+01 2.05e+02 8.70e+09  -1.0 6.29e+08    -  6.00e-01 1.16e-04h 12
1103  3.1271744e+01 2.05e+02

1187  7.9580544e+01 1.32e+06 5.60e+10  -1.0 1.94e+08    -  2.44e-02 9.78e-03w  1
1188  3.2597166e+01 2.13e+02 8.71e+09  -1.0 1.93e+08    -  6.37e-02 1.13e-04h 11
1189  3.2596739e+01 2.14e+02 8.71e+09  -1.0 6.25e+08    -  1.00e+00 2.26e-04h 11
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1190  3.2593905e+01 2.14e+02 8.71e+09  -1.0 6.25e+08    -  6.42e-02 2.26e-04h 11
1191  3.2588692e+01 2.15e+02 8.71e+09  -1.0 6.24e+08    -  5.77e-01 2.26e-04h 11
1192  3.2581097e+01 2.15e+02 8.71e+09  -1.0 6.24e+08    -  6.87e-02 2.26e-04h 11
1193  2.4625523e+01 1.33e+06 7.47e+10  -1.0 6.24e+08    -  6.88e-01 2.31e-01f  1
1194  2.8142469e+01 1.32e+06 6.40e+10  -1.0 2.12e+08    -  1.15e-02 5.70e-03h  2
1195  3.0016918e+01 1.32e+06 3.83e+10  -1.0 1.95e+08    -  3.31e-02 8.05e-04h  5
1196  3.0990094e+01 1.32e+06 2.81e+10  -1.0 1.95e+08    -  2.25e-02 2.39e-04h  8
1197  3.1471985e+01 1.32e+06 2.34e+10  -1.0 1.95e+08    -  2.29e-02 1.20e-04h  9
1198  3.1713278e+01 1.32e+06

1286 -7.9692325e+02 3.79e+01 3.03e+04  -1.0 1.88e+08    -  1.66e-02 9.26e-05h  1
1287 -7.9641330e+02 3.81e+01 9.91e+04  -1.0 1.88e+08    -  2.79e-02 1.06e-04h  7
1288 -7.9590976e+02 3.83e+01 2.53e+05  -1.0 1.88e+08    -  3.04e-02 1.05e-04h  7
1289 -7.9541257e+02 3.84e+01 8.45e+05  -1.0 1.88e+08    -  2.77e-02 1.03e-04h  7
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1290 -7.9492168e+02 3.86e+01 6.81e+06  -1.0 1.88e+08    -  5.64e-02 1.02e-04h  7
1291 -7.9443703e+02 3.88e+01 2.68e+07  -1.0 1.87e+08    -  1.99e-02 1.01e-04h  7
1292 -7.9395857e+02 3.89e+01 3.82e+08  -1.0 1.87e+08    -  8.62e-02 9.92e-05h  7
1293 -7.9348624e+02 3.91e+01 1.46e+09  -1.0 1.87e+08    -  1.80e-02 9.78e-05h  7
1294 -7.9301997e+02 3.93e+01 2.60e+10  -1.0 1.87e+08    -  1.06e-01 9.64e-05h  7
1295 -7.9256754e+02 3.94e+01 1.00e+11  -1.0 1.87e+08    -  1.77e-02 9.35e-05h  7
1296 -7.9212101e+02 3.95e+01 1.96e+12  -1.0 1.87e+08    -  1.13e-01 9.21e-05h  7
1297 -7.6393600e+02 6.09e+02

1421r-1.2597050e+02 6.67e+03 1.06e+03  -6.3 7.73e+07    -  1.00e+00 1.90e-08f  1
1422r-1.1856145e+02 6.95e+03 1.05e+03  -6.3 7.73e+07    -  5.85e-01 8.39e-03f  1
1423r-1.1855867e+02 6.95e+03 1.18e+03  -6.3 1.23e+07    -  1.00e+00 2.62e-06f  1
1424r-8.6856757e+00 5.41e+03 9.26e+02  -6.3 1.23e+07    -  6.30e-01 1.04e-01f  1
1425r-8.3227529e+00 5.41e+03 9.25e+02  -6.3 1.10e+07    -  1.00e+00 3.96e-04f  1
1426r 2.9368896e+02 5.60e+03 5.82e+02  -6.3 1.10e+07    -  6.42e-01 3.30e-01f  1
1427r 2.9368065e+02 5.59e+03 5.82e+02  -6.3 4.71e+02  -4.3 1.75e-06 3.35e-04h  1
1428r 2.9370622e+02 5.59e+03 5.87e+02  -6.3 7.23e+06    -  2.30e-01 6.01e-05f  1
1429r 2.9370622e+02 5.59e+03 6.71e+02  -6.3 7.49e+06    -  8.15e-01 1.40e-07h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1430r 3.3407153e+02 1.90e+04 2.11e+02  -6.3 7.50e+06    -  1.55e-06 7.88e-01f  1
1431r 3.3403478e+02 1.89e+04 1.80e+03  -6.3 9.99e+01  -4.8 9.33e-02 4.69e-03h  1
1432r 3.3403475e+02 1.89e+04

## Display Solution

In [35]:
#get value of surrogate inputs
x = [value(m.pmax),
    value(m.pmin_multi),
    value(m.ramp_multi),
    value(m.min_up_time),
    value(m.min_dn_multi),
    value(m.marg_cst),
    value(m.no_load_cst),
    value(m.startup_cst)
    ]

print("Market Inputs:", x)

#calculate revenues, costs, etc...
optimal_objective = -value(m.obj)

zone_hours = [value(m.zone_off.zone_hours)]
scaled_zone_hours = [value(m.zone_off.scaled_zone_hours)]
op_cost = [value(m.zone_off.fs.operating_cost)]
op_expr = value(m.zone_off.fs.operating_cost) # in dollars [$]
for zone in m.op_zones:
    zone_hours.append(value(zone.zone_hours))
    scaled_zone_hours.append(value(zone.scaled_zone_hours))
    op_cost.append(value(zone.fs.operating_cost))
    op_expr += value(zone.scaled_zone_hours)*value(zone.fs.operating_cost)

#more calculations of revenue and cost
revenue_per_year = value(m.revenue)
cap_expr = value(m.cap_fs.fs.capital_cost)/capital_payment_years
startup_expr = value(m.startup_expr)
total_cost = plant_lifetime*op_expr/1e6 + capital_payment_years*cap_expr
total_revenue = plant_lifetime*revenue_per_year

print("Annual Revenue [MM$]: ", revenue_per_year)
print("Capital Cost [MM$]: ",cap_expr)
print("Annual Startup Cost [MM$]: ", startup_expr)
print("Total Cost (20 years): ", total_cost)
print("Net Revenue (20 years)", optimal_objective)

Market Inputs: [177.5, 0.18002532733182175, 0.999999993111733, 4.0, 1.0, 21.58708760152878, 1.0, 61.09068702]
Annual Revenue [MM$]:  14.138371884753262
Capital Cost [MM$]:  65.6567774567005
Annual Startup Cost [MM$]:  3.969741156003134e-06
Total Cost (20 years):  639.0134881871701
Net Revenue (20 years) -356.24258117964735
